# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import numpy as np
import psycopg2
import pandas as pd
from sql_functions import *

In [2]:
db = Database()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
# all the files in song_data directory
data_files = get_files('data/song_data')

In [5]:
# example file for testing
first_file = data_files[0]

In [6]:
# import as Pandas dataframe
df = pd.read_json(first_file, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7SMBG1187B9B9066,NaN,NaN,,Los Manolos,SOBCOSW12A8C13D398,Rumba De Barcelona,218.38322,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
# extract song_data
song_data = df[["song_id", "title", "artist_id", "year", "duration"]].values[0]
song_data = list(song_data)
song_data

['SOBCOSW12A8C13D398',
 'Rumba De Barcelona',
 'AR7SMBG1187B9B9066',
 0,
 218.38322]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
db.execute_query(song_table_insert, song_data)

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].values[0]
artist_data = list(artist_data)
artist_data

['AR7SMBG1187B9B9066', 'Los Manolos', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
db.execute_query(artist_table_insert, artist_data)

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
log_files = get_files("data/log_data")
log_file = log_files[0]
log_file

'/home/shahin/Work/Learning/courses/data_engineering_nanodegree/1-data-modeling/project_1/data/log_data/2018/11/2018-11-27-events.json'

In [13]:
df = pd.read_json(log_file, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1543279932796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
1,Jimi Hendrix,Logged In,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Woodstock Inprovisation,200,1543280209796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
2,Building 429,Logged In,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Majesty (LP Version),200,1543280448796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
3,The B-52's,Logged In,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540870e+12,107,Love Shack,200,1543282396796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",38
4,Die Mooskirchner,Logged In,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540870e+12,107,Frisch und g'sund,200,1543282717796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",38


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df[df.page == "NextSong"]
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1543279932796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88


In [15]:
t = pd.to_datetime(df.ts / 1000, unit='s')
t

0     2018-11-27 00:52:12.796000004
1     2018-11-27 00:56:49.796000004
2     2018-11-27 01:00:48.796000004
3     2018-11-27 01:33:16.796000004
4     2018-11-27 01:38:37.796000004
                   ...             
298   2018-11-27 23:40:12.796000004
299   2018-11-27 23:42:55.796000004
300   2018-11-27 23:47:02.796000004
301   2018-11-27 23:51:36.796000004
302   2018-11-27 23:56:11.796000004
Name: ts, Length: 256, dtype: datetime64[ns]

In [16]:
time_data = dict(timestamp=df.ts, hour=t.dt.hour.values, day=t.dt.day.values,
                 weekofyear=t.dt.weekofyear.values, month=t.dt.month.values, 
                 year=t.dt.year.values, weekday=t.dt.weekday.values)
time_data = pd.DataFrame(data=time_data)
time_data

,timestamp,hour,day,weekofyear,month,year,weekday
0,1543279932796,0,27,48,11,2018,1
1,1543280209796,0,27,48,11,2018,1
2,1543280448796,1,27,48,11,2018,1
3,1543282396796,1,27,48,11,2018,1
4,1543282717796,1,27,48,11,2018,1
...,...,...,...,...,...,...,...
298,1543362012796,23,27,48,11,2018,1
299,1543362175796,23,27,48,11,2018,1
300,1543362422796,23,27,48,11,2018,1
301,1543362696796,23,27,48,11,2018,1


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [17]:
for i, row in time_data.iterrows():
    db.execute_query(time_table_insert, list(row))

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [18]:
user_df = df[["userId", "firstName", "lastName", "gender", "level"]]
user_df

,userId,firstName,lastName,gender,level
0,88,Mohammad,Rodriguez,M,paid
1,88,Mohammad,Rodriguez,M,paid
2,88,Mohammad,Rodriguez,M,paid
3,38,Gianna,Jones,F,free
4,38,Gianna,Jones,F,free
...,...,...,...,...,...
298,80,Tegan,Levine,F,paid
299,80,Tegan,Levine,F,paid
300,80,Tegan,Levine,F,paid
301,80,Tegan,Levine,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [19]:
for i, row in user_df.iterrows():
    db.execute_query(user_table_insert, row)

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [25]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    db.execute_query(song_select, (row.song, row.artist, row.length))
    results = db.cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId,\
                     row.location, row.userAgent)
    
    db.execute_query(songplay_table_insert, songplay_data)

cursor already closed


InterfaceError: cursor already closed

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [22]:
db.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.